<a href="https://colab.research.google.com/github/mzhkz/coefont_kana_convert_error_detector/blob/main/0827_detect_kana_yomi_error.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import requests as reqs
import json

## APIのアクセストークンなど機密情報の管理

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
secrets = {}
with open('/content/drive/MyDrive/coefont_kana_converter_error_detector/secrets.json') as f:
  secrets = json.load(f)

## 各種、依存記号の設定

In [ ]:
SYMBOL_READING_POINT = "、" # 句読点
SYMBOL_PUNCTUATION = "。" # 読点
SYMBOL_HALF_SPACE = " " # 半角スペース
SYMBOL_LONG_NOTE = "ー" # 伸ばし棒
SYMBOL_NONE = "" # 空文字

## 検査するCoeFontのかな変換出力と、元の文を入力する

In [ ]:
input = "モウネ、イイタイコトワ、タイトルデホボイッテシマッテイルノデスガ、ホンブンカキマス。ワタシッテ、タマニソーユートコアル。コレマデニモナンドカトージョーシテイルヨーニ、ワガヤノキッチンワセマイデス。ソンナセマイキッチンデモ、ジブンノタベタイモノオタベラレルヨーニ、テノコンダリョーリワシナイモノノ、クフーオコラシテイルツモリ。タトエバコンナノトカ。キョーワ、グタイテキナレシピデワナク、オヤクダチドーグニツイテカイテミマス。ソレガコチラ。ガゾーニハシオキタススプーンオキアルイワハシオキタスショーユザラトイッタトコロデショーカ。オハシオオクスペースノホカニ、フカメノスペースガアルノデ、タベルトキニツカウカトラリーガハシダケデワナイトキ、スープガアルカラスプーンモダシタイトユーシチュエーションヤ、チョーミリョーオ、リョーリトワ、ベツノトコロニダシタイケド、ワザワザ、ソノタメニオサラオツカウホドデワ、トユートキニベンリ。コノヨーニ、ワタシワ、タベルサイニツカウソーテーデ、コノハシオキオコーニューシタノデスガ、コレガ、ジツワ、ツクルサイニモカツヤクスルンデス。タトエバ、イタメブツオシテイルトキノサイバシ。ワタシワ、フカメノフライパン、ホボナベオツカッテイルタメ、ナカニイレタママデモ、キョクタンニハシガネソベルコトワナク、タショーアツクナッチャウカナークライノデメリットシカナイノデスガ、ソーワイッテモハシオカネツスルノワ、ハシノジュミョーオチジメテシマイマスヨネ。ソーオモッテフライパンカラハナソーニモ、キッチンガセマイトオキバガナイ。ハズカシナガラ、ナガシノブブンニハシサキガデル、ハシサキダケワダイニツカナイジョータイデ、ダイニチョクオキシテシマウトキモアリマス。キッチンデワ、テーネーナドーサデチョーリオスルナラ、ベツトシテ、アワタダシクリョーリシテイルト、マガタマミタイナチーサナハシオキニチャントハシオオクトユードーサガメンドーナノデスヨネ、コジンサガアルコトワミトメマス。ソンナトキニベンリダッタノガ、コノハシオキ。バツグンノアンテーカンガアリ、ハシオオクタメノミゾモナガメニカクホサレテイル。コレナラ、タショーナリトモ、ザツナオキカタオシタトテ、ハシガコロゲオチルコトワナイシ、ツギニツカウトキモテニトリヤスイデス。ソシテ、スプーンオキトシテモツカエルノデ、ニモノノアジミニツカッタスプーンオ、ソノツドアラッタリコザラニオイタリシナクテヨイ！コノヨーニ、チョーリチューノカトラリーオキトシテ、ダイカツヤクシテイルノデス。サラニ、フダンカラノートオミテクダサッテイルカタワ、ワタシガヨクオチャオノムコトオゴゾンジダトオモイマス。キチントイレルトキワチャバカラデスガ、ヤッパリテガルナティーバッグモソレナリニツカウワケデ。モーオワカリデショーカ、コノハシオキノ、スプーンオキノブブン、ティーバッグニピッタリナノデス。ガゾーイチオチャノシュルイニヨリマスガ、イチドキリデナクナンドカツカエルヨーナティーバッグガアリマスヨネ。ニイメノ、スコシシブイコーチャ、リョクチャヤ、ムシロ、ニイメイコーノホーガフーミノヒキタツウーロンチャナド。ソーユーオチャオイレタトキニ、コノハシオキノドーニューイゼンワ、コザラオダシテティーバッグオキバニシテイタノデスガ、イクラチーサクテモオサラワエンケーナノデスペースオトリマスシ、ティーバッグニワミズケガアルタメ、ヒラタイオサラデワココロモトナイバアイモアリマス。ソレガ、コノチョーホーケーノハシオキダトジャマニナラズ、フカサガアルブン、アンシンカンモオーキイデス。ティーバッグノイチジオキバトシテ、トテモベンリ。アト、コレワ、ホントーニ、シランガナッテイワレチャイソーナノデスガ、ハシオキミタイナチーサナモノッテ、アラッタアトカワカストキノオキバショニコマリマセンカ？ミズキリカゴノナカデ、コマカイモノワコノアタリニオク、トオーマカニキメテワイルノデスガ、ツギニツカオートオモッタラ、ユクエフメー、ナコトガトキドキアリマス。デモネ、コノハシオキダト、ミズキリチューモ、ソンザイカンガチャントアルノ。マー、ソモソモノサイズガオーキイトユーコトモアリマスガ、タンニソレダケノハナシデワナク、ウラガエスト、ハシオオクミゾノブブンオ、カゴノフチニヒッカケラレルノデ、イツノマニカイナクナッチャウコトガナイ。コレモマタ、ワタシニトッテワ、ヒジョーニ、オーキナメリットトナリマス。トユーヨーニ、コノハシオキニワ、キタイイジョーノカチガアリマシタ！セマイキッチンデウマクスゴスコツワ、モノオヘラスコトダトカンガエガチデスガ、ヤミクモニヘラスヒツヨーワナク、ベンリナモノオモツコトモオナジクライタイセツナノダトオモイマス。ナベオカイカエルコトガアッタラ、フタニハシオオケルタイプノモノオエラボーカナー。コレカラモ、ズイジ、モチモノオミナオシテイキタイデス。シチシノサンカク。"
original = "もうね、言いたいことはタイトルでほぼ言ってしまっているのですが、本文書きます。わたしってたまにそういうとこある。これまでにも何度か登場しているように、我が家のキッチンは狭いです。そんな狭いキッチンでも自分の食べたいものを食べられるように、手の込んだ料理はしないものの、工夫を凝らしているつもり。例えばこんなのとか。今日は、具体的なレシピではなく、お役立ち道具について書いてみます。それがこちら。箸置き＋スプーン置きあるいは箸置き＋しょうゆ皿といったところでしょうか。お箸を置くスペースのほかに、深めのスペースがあるので、食べるときに使うカトラリーが箸だけではないとき（スープがあるからスプーンも出したいというシチュエーション）や、調味料を料理とは別のところに出したいけどわざわざそのためにお皿を使うほどでは…というときに便利。このように、わたしは「食べる際に使う」想定でこの箸置きを購入したのですが、これが実は「作る際にも」活躍するんです。例えば、炒め物をしているときの菜箸。わたしは深めのフライパン（ほぼ鍋）を使っているため、中に入れたままでも極端に箸が寝そべることはなく、多少熱くなっちゃうかなぁくらいのデメリットしかないのですが、そうは言っても箸を加熱するのは箸の寿命を縮めてしまいますよね。そう思ってフライパンから離そうにも、キッチンが狭いと置き場がない。恥ずかしながら、流しの部分に箸先が出る（箸先だけは台につかない）状態で、台に直置きしてしまうときもあります…。キッチンでは、丁寧な動作で調理をするなら別として、あわただしく料理していると「勾玉みたいな小さな箸置きにちゃんと箸を置く」という動作が面倒なのですよね（個人差があることは認めます）。そんなときに便利だったのが、この箸置き。抜群の安定感があり、箸を置くための溝も長めに確保されている。これなら多少なりとも雑な置き方をしたとて、箸が転げ落ちることはないし、次に使うときも手に取りやすいです。そして、スプーン置きとしても使えるので、煮物の味見に使ったスプーンを、その都度洗ったり小皿に置いたりしなくて良い！このように、調理中のカトラリー置きとして、大活躍しているのです。さらに、普段からnoteを見てくださっているかたは、わたしがよくお茶を飲むことをご存じだと思います。きちんといれるときは茶葉からですが、やっぱり手軽なティーバッグもそれなりに使うわけで。もうお分かりでしょうか…この箸置きの「スプーン置き」の部分、ティーバッグにぴったりなのです。お茶の種類によりますが、一度きりでなく何度か使えるようなティーバッグがありますよね。２煎目の少し渋い紅茶・緑茶や、むしろ２煎目以降のほうが風味の引き立つ烏龍茶など。そういうお茶をいれたときに、この箸置きの導入以前は、小皿を出してティーバッグ置き場にしていたのですが、いくら小さくてもお皿は円形なのでスペースをとりますし、ティーバッグには水気があるため、平たいお皿では心許ない場合もあります。それが、この長方形の箸置きだとじゃまにならず、深さがある分、安心感も大きいです。ティーバッグの一時置き場として、とても便利。あと、これはほんとうに「知らんがな」って言われちゃいそうなのですが、箸置きみたいな小さなものって、洗ったあと乾かすときの置き場所に困りませんか？水切りかごの中で、細かいものはこのあたりに置く、と大まかに決めてはいるのですが、次に使おうと思ったら行方不明…なことがときどきあります。でもね、この箸置きだと、水切り中も存在感がちゃんとあるの。まあそもそものサイズが大きいということもありますが、単にそれだけの話ではなく、裏返すと、箸を置く溝の部分をかごのフチに引っかけられるので…いつの間にかいなくなっちゃうことがない。これもまた、わたしにとっては非常に大きなメリットとなります。というように、この箸置きには期待以上の価値がありました！狭いキッチンでうまく過ごすコツは「物を減らすこと」だと考えがちですが、やみくもに減らす必要はなく、「便利な物を持つこと」も同じくらい大切なのだと思います。鍋を買い替えることがあったら、ふたに箸を置けるタイプのものを選ぼうかな～。これからも随時、持ち物を見直していきたいです。七志野さんかく"

## 前処理や記号を付与するためのクラス

Coefontの出力(subject)と、オリジナルソースを管理するクラス。比較に必要な前処理などを行う。

In [ ]:
class Subject:
  def __init__(self, original, subject):
    self.subject = subject
    self.original = original

    self.symbolTrigger1 = [
        ["オ", "コ", "ソ", "ト", "ノ", "ホ", "モ", "ヨ", "ロ", "ヲ"],
        ["エ", "ケ", "セ", "ネ", "ヘ", "メ", "レ"], # removed テ
        ["ア", "カ", "サ", "タ", "ナ", "ハ", "マ", "ラ", "ヤ", "ラ", "ワ"]
        ]

    self.symbolTrigger2 = [
        ["ウ"],
        ["イ"],
        ["ア"]
        ]

    sentences_subject, sentences_original = self.preprosessing(self.subject), self.preprosessing(self.original),

    self.sentences_subject = sentences_subject
    self.sentences_original = sentences_original


  def preprosessing(self, subject):
    handled = self._replaceSymbol(subject) #ここから文字列
    handled = self.revertSymbol(handled)  # 不規則な伸ばし変換があるので、オリジナルソースを伸ばし変換するのではなくCoefontの伸ばし変換を逆作用させることにした
    handled = self._splitSentence(handled) # ここから配列

    return handled

  def _splitSentence(self, subject):
    sentences = subject.split(SYMBOL_PUNCTUATION)
    return sentences

  def _replaceSymbol(self, subject):
    sentence = subject.replace(SYMBOL_READING_POINT, SYMBOL_NONE)
    return sentence

  def convertSymbol(self, katakana_original):
    list_kat_original = list(katakana_original)
    kat_original_size = len(list_kat_original)-1
    idkc = 0
    while  idkc < kat_original_size:
      kat_char_pointer = list_kat_original[idkc]
      kat_char_next = list_kat_original[idkc+1]
      for ids, symbols in enumerate(self.symbolTrigger1):
        if kat_char_pointer in symbols and kat_char_next in self.symbolTrigger2[ids]:
          list_kat_original[idkc+1] = SYMBOL_LONG_NOTE
          idkc+=1
      idkc+=1
    return "".join(list_kat_original)

  def revertSymbol(self, katakana_subject):
    list_kat_subject = list(katakana_subject)
    kat_subject_size = len(list_kat_subject)-1
    idkc = 0
    while  idkc < kat_subject_size:
      kat_char_pointer = list_kat_subject[idkc]
      kat_char_next = list_kat_subject[idkc+1]
      if kat_char_next == SYMBOL_LONG_NOTE:
         for ids, symbols in enumerate(self.symbolTrigger1):
           if kat_char_pointer in symbols:
               list_kat_subject[idkc+1] = self.symbolTrigger2[ids][0]
               idkc+=1
      idkc+=1
    return "".join(list_kat_subject)

## 棒読みの変換

In [ ]:
test01_subject = Subject(original, input)
# print(test01_subject.convertSymbol("コウコウセイ"))
# print(test01_subject.convertSymbol("ケイト"))
# print(test01_subject.convertSymbol("オカアサン"))
# print(test01_subject.convertSymbol("ヤッテミヨウ"))

# モウネ イイタイ コトハ タイトルデ ホボ イッテシマッテイルノデスガ ホンブン カキマス
test01_original = "モウネ イイタイ コトハ タイトルデ ホボ イッテシマッテイルノデスガ ホンブン カキマス"
test01_converted = test01_subject.convertSymbol(test01_original)
print("original:", test01_original)
print("converted:", test01_converted)
print("reverted:", test01_subject.revertSymbol(test01_converted))
print("match (equal between original and reverted):", test01_subject.revertSymbol(test01_converted) == test01_original)

original: モウネ イイタイ コトハ タイトルデ ホボ イッテシマッテイルノデスガ ホンブン カキマス
converted: モーネ イイタイ コトハ タイトルデ ホボ イッテシマッテイルノデスガ ホンブン カキマス
reverted: モウネ イイタイ コトハ タイトルデ ホボ イッテシマッテイルノデスガ ホンブン カキマス
match (equal between original and reverted): True


## APIやライブラリごとにかな変換と比較を行う


エラー検出方法1 全体像
1.   対象、オリジナルともに読点をすべて空白に置き換える（削除する）
2.   対象、オリジナルともに句読点ごとにセンテンスを区切る
3.   オリジナルのセンテンスをカタカナ変換する。なお結果をキーワード（一定の塊）ごとに分割する。
4.   対象の箇所をシンボルに置き換える。(a行+a行の場合、「ー」にあたる補完ができなくなる。不可逆性があるため、オリジナルの対象個所を伸ばしに置き換える)
5.   両者を比較する。一致しなかった場合 (6)へ
6.   キーワードごとに誤字を検出する。キーワードと対象の開始地点をそろえ、一文字ずつ頭から比較していく。一致しない箇所があれば（７）に進む
7.   一致しない文字に出会ったら、次のキーワードに出会うまで探索を続けて、キーワードの頭文字に出会ったら、その場所以前の内容をエラーとして記録する。完了後は6を繰り返す。



## キーワードとCoefontの出力の比較

### 想定される誤字ケース


*   「たいいくかん」to 「たんいくかん」、、文字数は同じだか文字が違う
*   「たいいくかん」to 「たいくかん」、、文字数が違う（少ない）
*   「たいいくかん」to 「たいいいくかん」、、文字数が（多い）



### 実装
とりあえず原始的な方法で実装 **(version1)**
できること


*  **文字数が同じ**とき、**文字が異なる場合（エラー）を検知**でき、かつ**キーワードごとにエラーを取り出す**ことができる


想定されるケースを踏まえて実装 **(version2)**
できること


*  **文字数が違う**とき、**文字が異なる場合（エラー）を検知**でき、かつ**キーワードごとにエラーを取り出す**ことができる
* キーワードが2文字以上ではなくてはならない





In [ ]:
## implemented with version2

# test02_subject = list("モウネイイタイコトワタイトルデホボイッテシマッテイルノデスガホンブンカキマス")
# test02_subject = list("モウネイXイタイコトワタイトルデホボイッテシマッテイルノデスガホンブンカキマス")
test02_subject = list("モウネイイタイタイイイクタイトルデホボイッテシマッテイルノデスガホンブンカキマス")
test02_keywords = ['モウネ', 'イイタイ', 'タイイク', 'タイトルデ', 'ホボ', 'イッテシマッテイルノデスガ', 'ホンブン', 'カキマス']


idk = 0
ids = 0

test02_err = {}
test02_subject_size = len(test02_subject)
test02_keyword_size = len(test02_keywords)

while idk < test02_keyword_size-1:
  pointed_keyword = test02_keywords[idk]
  rshift_keyword =  test02_keywords[idk+1]
  keyword_size = len(pointed_keyword)
  frame_pointrer = ids+keyword_size
  clip = test02_subject[ids:frame_pointrer]
  for idc, char in enumerate(clip):
    if char == pointed_keyword[idc] or char == SYMBOL_LONG_NOTE:
      continue
    else:
      buffer = []
      test02_err[pointed_keyword] = "".join(clip)
      # つぎの開始地点を見つける
      for idn in range(ids, test02_subject_size):
        if len(buffer) < 2:
          buffer = buffer + [test02_subject[idn]]
        else:
          buffer[0] = buffer[1] # left shift
          buffer[1] = test02_subject[idn] # append
          if SYMBOL_NONE.join(buffer) == rshift_keyword[:2]:
            test02_err[pointed_keyword] = "".join(test02_subject[ids:idn-1])
            ids += (idn-ids-keyword_size-1) # offsetを足す
            break
      break
  idk+=1
  ids+=keyword_size

print("errors:", test02_err)

errors: {'タイイク': 'タイイイク'}


# 以下、クラス実装

In [ ]:
class CompareSentence:
  def __init__(self, subject, converters=[]):
    self.subject = subject
    self.converters = converters
    self.converted = [[ [] for ids in range(len(self.subject.sentences_original))] for idc in range(len(self.converters))] # キーワードが入る。

  def convert(self, idc, ids):
    converter = self.converters[idc]
    sentence = self.subject.sentences_original[ids]
    converted = converter.convert(sentence)
    # converted = subject.convertSymbol(converted) # 不規則な伸ばし変換があるので、オリジナルソースを伸ばし変換するのではなくCoefontの伸ばし変換を逆作用させることにした
    keywords = converted.split(SYMBOL_HALF_SPACE) # wordごと（半角空白を境）に区切る　区切られたものをキーワードとする
    
    self.converted[idc][ids] = keywords

  def convertAll(self, idc):
    for ids in range(len(self.subject.sentences_original)):
      self.convert(idc, ids)

  # version 1
  def compare(self, idc, ids):
    converted_keyword = self.converted[idc][ids]
    subject = self.subject.sentences_subject[ids]
    idk = 0
    ids = 0
    err = {}
    while idk < len(converted_keyword):
      keyword = converted_keyword[idk]
      keyword_size = len(keyword)
      frame_pointrer = ids+keyword_size
      clip = subject[ids:frame_pointrer]
      for idc, char in enumerate(clip):
        if char == keyword[idc]:
          continue
        else:
          err[keyword] = SYMBOL_NONE.join(clip)
          break
      idk+=1
      ids+=keyword_size
    return err


# 以下、コンバーターのクラス実装
各クラスでは`convert()`関数の実装がmustです。CompareSentenceクラスでは、この関数を実行して、各APIに差分をボックス化しています。

## PyKakashiLib

In [ ]:
class PyKakashiConverter:
  def __init__(self):
    pass

  def convert():
    pass

### Goo API

ドキュメントはこちら
https://labs.goo.ne.jp/api/jp/hiragana-translation/

In [ ]:
class GooConverter:
  def __init__(self, app_id):
    self.app_id = app_id

  def convert(self, sentence):
    url = "https://labs.goo.ne.jp/api/hiragana"
    params = {
        "app_id": self.app_id,
        "sentence": sentence,
        "output_type": "katakana"
    }

    headers = {'Content-Type': 'application/json'}

    r = reqs.post(url=url, data=params)
    response_text = r.text
    print(response_text)
    response_json = json.loads(response_text)
    return response_json["converted"]

### Yahoo API(未完成)

ドキュメントはこちら
https://developer.yahoo.co.jp/webapi/jlp/furigana/v2/furigana.html

In [ ]:
class YahooConverter:
  def __init__(self, app_id=""):
    self.app_id = app_id

  def convert(self, sentence):
    url = "https://jlp.yahooapis.jp/FuriganaService/V2/furigana"
    params = {
        "id": "",
        "jsonrpc": "2",
        "method": "jlp.furiganaservice.furigana",
        "params": {
            "q": sentence,
        }
    }

    headers = {'Content-Type': 'application/x-www-form-urlencoded'}

    r = reqs.post(url=url, data=params, headers=headers)
    response_text = r.text
    print(response_text)
    response_json = json.loads(response_text)
    kana_list = [word["furigana"] for word in response_json["word"]]
    return SYMBOL_HALF_SPACE.join(kana_list)

## Goo APIを利用したオリジナルソースのかな変換



*   Subjectクラスを初期化して、Coefontの出力ともとのオリジナルソースを引き渡す（センテンス単位の分割など、下処理を行う）
*   オリジナルソースのかな変換を行う比較対象のコンバータを初期化する。コンバータは複数利用されることが想定されており、優先的に使うコンバータから配列形式で記述する
*   CompareSentenceクラスを初期化して、先ほど初期化したSubjectとコンバータを引き渡す。





In [ ]:
subject = Subject(original, input) 
converter_0 = GooConverter(app_id=secrets["goo_api_id"]) # goo apiの初期化
converter_1 = YahooConverter()

converters = [
    converter_0,  ## goo apiを優先的に使う
    converter_1
  ]
compare = CompareSentence(subject, converters)


xy = [0, 0] # converter id, sentence id
compare.convertAll(xy[0]) # converter_idが0のconverterを利用してsentence_idが0の文章をかな変換する
print("sentence size:", len(compare.subject.sentences_original), len(compare.subject.sentences_subject))
print("original sentense:", compare.subject.sentences_original[xy[1]])
print("convert sentense:", SYMBOL_NONE.join(compare.converted[xy[0]][xy[1]])) # converter_idが0のconverterを利用してsentence_idが0の文章をかな変換した結果を取得する
print("subject sentense:", compare.subject.sentences_subject[xy[1]])

{"converted": "モウネ イイタイ コトハ タイトルデ ホボ イッテシマッテイルノデスガ ホンブン カキマス", "output_type": "katakana", "request_id": "labs.goo.ne.jp\t1661658327\t0"}
{"converted": "ワタシッテ タマニ ソウイウ トコ アル", "output_type": "katakana", "request_id": "labs.goo.ne.jp\t1661658327\t0"}
{"converted": "コレマデニモ ナンドカ トウジョウシテイルヨウニ ワガヤノ キッチンハ セマイデス", "output_type": "katakana", "request_id": "labs.goo.ne.jp\t1661658328\t0"}
{"error": {"code": 400, "message": "Invalid app_id"}}


KeyError: ignored

### （作業用）キーワードごとに分けられたセンテンスを取得

In [ ]:
print(compare.converted[xy[0]][xy[1]])

['モウネ', 'イイタイ', 'コトハ', 'タイトルデ', 'ホボ', 'イッテシマッテイルノデスガ', 'ホンブン', 'カキマス']
